# Connect to Indy by IndyDCP

* **robot_ip**: IP address of robot (actually, STEP in IndyCB)
* **name**: string represents robot model
    * Indy7: "NRMK-Indy7"
    * IndyRP2: "NRMK-IndyRP2"
    

* 1.Setting   
* 2.Connect & Disnonnect
* 3.Get robot status

In [1]:
from indy_utils import indydcp_client as client
from indy_utils.indy_program_maker import JsonProgramComponent

import json
import threading
from time import sleep

# Set robot (server) IP 
robot_ip = "10.201.159.232"  # Robot (Indy) IP

# Set robot name
name = "NRMK-Indy7"  # Robot name (Indy7)
# name = "NRMK-IndyRP2"  # Robot name (IndyRP2)

# Create class object
indy = client.IndyDCPClient(robot_ip, name)

In [2]:
# Connect to robot
indy.connect()

# Disconnect with robot
# recommanded for finishing program properly
indy.disconnect()

Connect: Server IP (10.201.159.232)


In [3]:
indy.connect()

status = indy.get_robot_status()
print(status)

indy.disconnect()

Connect: Server IP (10.201.159.232)
{'emergency': 0, 'collision': 0, 'zero': 1, 'ready': 1, 'home': 0, 'busy': 0, 'direct_teaching': 0, 'error': 0, 'teaching': 0, 'movedone': 1, 'resetting': 0}


# Examples of JSON Program maker using extended IndyDCP
## Indy7 control code (Indy7 Control Code)


### Get position data
* __indy.get_joint_pos()__  : get current joint angle (degree)
* __indy.get_task_pos()__  : get current task-space pose (meter, and degree)

In [4]:
indy.connect()

print(indy.get_joint_pos())
print(indy.get_task_pos())

indy.disconnect()

Connect: Server IP (10.201.159.232)
[-0.00027238830061985267, 0.0019067181043389686, 0.001225747352789337, 0.001196530785891168, 0.0044054088025993, -0.00021755105198021236]
[-6.372758333837825e-05, -0.1864999999128923, 1.3279999973200038, -2.873509581650046e-08, -0.007674068409596878, 0.0005070198115096626]


### Joint, Task Control (Joint, Task 제어하기)

* **indy.go_home()** : go to home position
* **indy.go_zero()** : go to zero position
* **indy.joint_move_to(joint_pos)** : add waypoint using joint control - Absolute point (절대값) 
* **indy.joint_move_by(joint_pos)** : add waypoint using joint control - Relativ point (상대값) 
* **indy.task_move_to(task_pos)** : add waypoint using task control - Absolute point (절대값) 
* **indy.task_move_by(task_pos)** : add waypoint using task control - Relativ point (상대값) 

In [ ]:
## Zero, Home pose

indy.connect()

indy.go_zero()
#indy.go_home()

indy.disconnect()

In [ ]:
##Joint 제어

indy.connect()

j_pos1 = indy.get_joint_pos()
print("j_pos1", j_pos1)


#j_pos1[1] += 20  # move for Indy7
#indy.joint_move_to(j_pos1)  # Move 6th joint

j_pos_rel = [10, -10, 0, 0, 0, 20]  # move for Indy7
indy.joint_move_by(j_pos_rel)


indy.disconnect()

In [ ]:
##Task 제어

indy.connect()

t_pos1 = indy.get_task_pos()
print("t_pos1", t_pos1)

##t_pos1[2] += 1
##indy.task_move_to(t_pos1)  # Move along z-axis

t_pos_rel = [1, -2, 3, 0, 0, 0]
indy.task_move_by(t_pos_rel)

indy.disconnect()

### Make Trajectory
* **prog = JsonProgramComponent(policy=0, resume_time=2)** : setting Trajectory data
    * policy: collision policy (0: keep pause, 1: resume after 'resume_time', 2: stop program, 3: no collision detection)
    
    
* **prog.add_move_home()** : go to home position
* **prog.add_move_zero()** : go to zero position
* **prog.add_joint_move_to(joint_pos)** : add waypoint using joint control - Absolute point (절대값) 
* **prog.add_joint_move_by(joint_pos)** : add waypoint using joint control - Relativ point (상대값) 
* **prog.add_task_move_to(task_pos)** : add waypoint using task control - Absolute point (절대값) 
* **prog.add_task_move_by(task_pos)** : add waypoint using task control - Relativ point (상대값)


* **prog.add_digital_out(idx=idx, val=0)** : joint, task를 구분하기 위한 digital bit 설정 (val 값 : joint = 0, task = 1)


* **prog_json = prog.program_done()**  : finish to make program, and return JSON string to be sent to 'indy'
* **indy.set_and_start_json_program(prog_json)** : start the made program
* **indy.set_and_start_json_program(prog.program_done())**  : more simple way

### Precautions (주의사항)

* Trajectory를 만들때는 joint와 task를 구분해서 써야 한다.
(한 Trajectory 상에 joint와 task를 섞어쓰기 위해서는 digital bit를 바꿔줘야 한다.)
* add_move_home(), add_move_zero()는 joint제어로 분류된다.

In [ ]:
## Home, Zero 반복이동

indy.connect()

indy.set_joint_vel_level(3)

prog = JsonProgramComponent(policy=1, resume_time=2)  # Init. prgoram

prog.add_move_home()
prog.add_move_zero()
prog.add_move_home()
prog.add_move_zero()

prog_json = prog.program_done()    # Program end

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

In [ ]:
## Joint Trajectory config (Joint 제어)

indy.connect()

# add_joint_move(pos)
# add_task_move(pos)

vel = 5
blend = 20

j_pos1 = [0.00222450445506213, -34.99499612603537, -60.00065070538877, -0.000652653155940637, -85.00062243773765, -0.005275613010520149]
j_pos2 = [0.7399428186338298, -87.25655043421288, 24.449029087036738, -3.217797609839321, -118.97317716391564, -0.0044054088025993]
j_pos3 = [0.6791548295454993, -73.52736291807045, -35.52456438049046, 3.6427292021196704, 6.2422467222312275, -0.004351021039604247]
j_pos4 = [-0.9590792064825013, -71.96308230566073, -34.78080812564795, 9.005471408957895, 55.15359708578643, -0.0036439801206685565]

prog = JsonProgramComponent(policy=1, resume_time=3)  # Init. prgoram

prog.add_joint_move_to(j_pos1, vel=vel, blend=blend)
prog.add_joint_move_to(j_pos2, vel=vel, blend=blend)
prog.add_joint_move_to(j_pos3, vel=vel, blend=blend)
prog.add_joint_move_to(j_pos4, vel=vel, blend=blend)

prog_json = prog.program_done()

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

In [ ]:
## Task Trajectory config (Task 제어)

indy.connect()

vel = 5
blend = 20

t_wp1 = [0.6910136725626225, -0.2123301970500489, 0.2583231171384601, -177.7962365761575, 0.9889072041064573, 179.2646142500768]
t_wp2 = [0.9564666192625594, -0.2921623813989345, 0.4357075789683925, 67.59249069789438, -173.05076991945145, 77.77963364441038]
t_wp3 = [0.7009320249592459, -0.24506788301261087, 0.3748936800814323, 2.5559736667629336, -178.07944812200918, 53.8743852989464]
t_wp4 = [0.5597762859029639, -0.20633576407169446, 0.3283168775628519, -1.464505932654421, -179.1338746927122, 58.01384472142377]

prog = JsonProgramComponent(policy=0, resume_time=2)                
prog.add_task_move_to(t_wp1, vel=vel, blend=blend)
prog.add_task_move_to(t_wp2, vel=vel, blend=blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=blend)

prog_json = prog.program_done()
indy.set_and_start_json_program(prog_json)

indy.disconnect()

In [ ]:
# mix trajectory config (joint and task)
indy.connect()

vel = 5
j_blend = 20
t_blend = 0.2

t_wp1 = [0.6067674966068215, -0.18649384650008538, 0.4100259354264202, 179.9982122370174, -0.009843770313927431, 179.99871942052945]
t_wp2 = [0.713555177030334, -0.010827591112717484, 0.2858938880590304, -5.836662073283785, 162.80757095440694, 8.9730407449388]
t_wp3 = [0.8071902738781609, -0.13292912970174506, 0.23895694055448075, -1.356536922079839, 153.53673284803128, 2.539941694447069]
t_wp4 = [0.6552232283948026, -0.30083843519838194, 0.3082318964747058, -172.7388071704742, 0.42433374155690934, 174.41714512772282]

j_wp1 = [-0.0, -35.005392279509024, -60.004146355246725, 0.001740408415841699, -85.00051366221166, 0.0011421430228961149]
j_wp2 = [12.637455207258064, -41.954608002972805, -67.10617191535825, 5.899059937732443, -53.56841134553262, 0.0011421430228961149]
j_wp3 = [3.565835243414491, -54.48969060724791, -54.60509245061052, 1.2845301864171639, -44.431049611311714, 0.0011421430228961149]
j_wp4 = [-7.420765269886853, -49.14193649922844, -54.59564965618903, -7.475978738011012, -75.72152619315165, 0.0010877552599010618]

prog = JsonProgramComponent(policy=0, resume_time=2)

prog.add_move_home()  
prog.add_joint_move_to(j_wp1, vel=vel) 

# Turns on digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=1)

# Wait for set time
prog.add_wait(1)

# Tool command of tool ID and its command
# Tool should be first set up in Conty Application (Setting - Tool)
# In Conty,add tool and set application (e.g. Pick & Place)
# Edit Hold and Release Output and update the tool (refer the Indy manual)
prog.add_endtool_do(type=0, value=0)
prog.add_wait(1)
prog.add_endtool_do(type=0, value=1)

prog.add_task_move_to(t_wp2, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=t_blend)

# Turns off digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=0)

prog.add_stop()  # Stop program

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()